In [4]:
import numpy as np
import pandas as pd
import xarray as xr
import dask
import os, sys
import glob
import zarr
from joblib import Parallel, delayed
import os

sys.path.append('/')
from libraries import *

dates = pd.date_range(start='2011-01-01T00', end='2020-12-31T23', freq='h')
zarr_store = '/media/harish/External_3/CERRA_wind_profiles_and_Chebyshev_coefficients/CERRA_height_level_winds.zarr'

- My initial idea has been to combine wind speed data at analysis and forecast times.
- However, I mistakenly downloaded analysis data for remaining vertical levels, rather than forecast. 
- I can still download the forecast data, but it takes too much time. 
- Thus, I first combine the analysis data, across height levels. 

In [5]:
print("Starting parallel computing...")
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=48,threads_per_worker=1,memory_limit='4GB',dashboard_address='8787')
# Connect to the cluster
client = dd.Client(cluster)
print(client)

Starting parallel computing...
<Client: 'tcp://127.0.0.1:42645' processes=48 threads=48, memory=178.81 GiB>


In [3]:
def preprocess(ds):
    ds['time'] = ds['valid_time']
    ds = ds.drop(['valid_time', 'step', 'latitude','longitude'])
    return ds
def preprocess_2(ds):
    '''
    This script process the remaining height level forecast data
    '''
    ds = ds.rename({'valid_time':'time'})
    ds = ds.drop(['expver','latitude','longitude'])
    return ds

# Initializing a zarr by reading sample data
- Once created, no need to repeat again.

In [5]:
def template_zarr_init(zarr_store, dates):
    ds = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500/2020/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_2020_1.nc').ws
    ds = preprocess(ds)
    template = ds.chunk({'time': 1,'y': -1,'x': -1, 'heightAboveGround':1}).pipe(xr.zeros_like).isel(time=0,heightAboveGround=0,
                                                                        drop=True).expand_dims(time=len(dates), heightAboveGround=len(CERRA_levels))
    template['time'] = dates
    template['heightAboveGround'] = CERRA_levels
    template = template.chunk({'time': 1,'heightAboveGround':1})
    template.to_dataset(name = 'wind_speed').to_zarr(zarr_store, compute=False, consolidated=True, mode='w')
    return template
'''
Initialize the zarr store, which creates the zarr store in disk, with zeros. 
Once created, better to chose append mode for further operations or else it will overwrite the existing data.
'''
template_zarr_init(zarr_store, dates)

<xarray.DataArray 'ws' (time: 87672, heightAboveGround: 12, y: 1069, x: 1069)>
dask.array<rechunk-merge, shape=(87672, 12, 1069, 1069), dtype=float32, chunksize=(1, 1, 1069, 1069), chunktype=numpy.ndarray>
Coordinates:
  * time               (time) datetime64[ns] 2011-01-01 ... 2020-12-31T23:00:00
  * heightAboveGround  (heightAboveGround) float64 10.0 15.0 ... 400.0 500.0
Dimensions without coordinates: y, x
Attributes: (12/33)
    GRIB_paramId:                             10
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      1142761
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                Wind speed
    GRIB_shortName:                           ws
    GRIB_units:                               m s**-1
    long_name:                                Wind speed
    units:                                    m s**-1
    standard_name:                            unknown

In [6]:
def read_monthly_data(year,month):
    chunks = {'time': 8}
    ds_10m = xr.open_dataset(f'/media/harish/External_3/CERRA_ws10/{year}/CERRA_{year}_{month}.nc',chunks = chunks).si10
    ds_10m = preprocess(ds_10m)
    ds_10m_1 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws10_step1/{year}/CERRA_{year}_{month}.nc',chunks = chunks).si10
    ds_10m_1 = preprocess(ds_10m_1)
    ds_10m_2 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws10_step2/{year}/CERRA_{year}_{month}.nc',chunks = chunks).si10
    ds_10m_2 = preprocess(ds_10m_2)

    ds_100m = xr.open_dataset(f'/media/harish/External_3/CERRA_ws100/{year}/CERRA_gridded_100_m_wind_{year}_{month}.nc',chunks = chunks).ws
    ds_100m = preprocess(ds_100m)
    ds_150m = xr.open_dataset(f'/media/harish/External_3/CERRA_ws150/{year}/CERRA_gridded_150_m_wind_{year}_{month}.nc',chunks = chunks).ws
    ds_150m = preprocess(ds_150m)
    ds_100_150m_1 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_100_150_step1/{year}/CERRA_gridded_wind_{year}_{month}_1.nc',chunks = chunks).ws
    ds_100_150m_1 = preprocess(ds_100_150m_1)
    ds_100_150m_2 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_100_150_step2/{year}/CERRA_gridded_wind_{year}_{month}_2.nc',chunks = chunks).ws
    ds_100_150m_2 = preprocess(ds_100_150m_2)

    ds_remaining_height = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500/{year}/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_{year}_{month}.nc',chunks = chunks).ws
    ds_remaining_height = preprocess(ds_remaining_height)
    chunks = {'valid_time':8}
    ds_remaining_height_1 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500_step1/{year}/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_{year}_{month}_1.nc',chunks = chunks).ws
    ds_remaining_height_1 = preprocess_2(ds_remaining_height_1)
    ds_remaining_height_2 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500_step2/{year}/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_{year}_{month}_2.nc',chunks = chunks).ws
    ds_remaining_height_2 = preprocess_2(ds_remaining_height_2)
    # Return a dictionary of datasets
    return {
        "10m": ds_10m,
        "10m_step1": ds_10m_1,
        "10m_step2": ds_10m_2,
        "100m": ds_100m,
        "150m": ds_150m,
        "100_150m_step1": ds_100_150m_1,
        "100_150m_step2": ds_100_150m_2,
        "remaining_height": ds_remaining_height,
        "remaining_height_step1": ds_remaining_height_1,
        "remaining_height_step2": ds_remaining_height_2,
    }

In [7]:


def write_chunk(ds_chunk, zarr_store, region):
    """
    Function to write a single chunk to the Zarr store.
    """
    ds_chunk.to_zarr(zarr_store, region=region, mode="r+")

def write_to_zarr_parallel(ds, zarr_store, n_jobs=os.cpu_count()):
    """
    Writes the dataset to the Zarr store in parallel using joblib.
    """
    # Determine the time and height indices
    time_indices = np.searchsorted(dates.values, ds.time.values)
    height_indices = np.atleast_1d(np.searchsorted(CERRA_levels, ds.heightAboveGround.values))

    # List to store all tasks
    tasks = []

    # Iterate over time and height indices to create tasks
    for i, time_idx in enumerate(time_indices):
        if len(height_indices) > 1:
            # Multiple height levels
            for j, height_idx in enumerate(height_indices):
                # Define the region
                region = {
                    "time": slice(time_idx, time_idx + 1),
                    "heightAboveGround": slice(height_idx, height_idx + 1),
                }

                # Create the data chunk
                ds_chunk = (
                    ds.isel(time=i, heightAboveGround=j)
                    .expand_dims(dim={"time": [ds.time.values[i]],
                                      "heightAboveGround": [ds.heightAboveGround.values[j]]})
                    .to_dataset(name="wind_speed")
                )

                # Add the task
                tasks.append(delayed(write_chunk)(ds_chunk, zarr_store, region))
        else:
            # Single height level
            region = {
                "time": slice(time_idx, time_idx + 1),
                "heightAboveGround": slice(height_indices[0], height_indices[0] + 1),
            }

            ds_chunk = (
                ds.isel(time=i)
                .expand_dims(dim={"time": [ds.time.values[i]],
                                  "heightAboveGround": [ds.heightAboveGround.values]})
                .to_dataset(name="wind_speed")
            )

            # Add the task
            tasks.append(delayed(write_chunk)(ds_chunk, zarr_store, region))

    # Run all tasks in parallel
    with Parallel(n_jobs=n_jobs, verbose=10) as parallel:
        parallel(tasks)

In [10]:
for year in range(2011,2020): # since 2020 is already processed
    for month in range(1,13):
        ds_stores = read_monthly_data(year,month)
        for key in ds_stores.keys():
            print(f"Writing {key} data to Zarr store...")
            ds = ds_stores[key].load()
            write_to_zarr_parallel(ds, zarr_store)

Writing 10m data to Zarr store...


[Parallel(n_jobs=128)]: Using backend LokyBackend with 128 concurrent workers.
[Parallel(n_jobs=128)]: Done   1 out of   1 | elapsed:    5.8s finished
[Parallel(n_jobs=128)]: Done  18 out of 248 | elapsed:    6.5s remaining:  1.4min
[Parallel(n_jobs=128)]: Done  43 out of 248 | elapsed:    6.6s remaining:   31.4s
[Parallel(n_jobs=128)]: Done  68 out of 248 | elapsed:    6.6s remaining:   17.6s
[Parallel(n_jobs=128)]: Done  93 out of 248 | elapsed:    6.7s remaining:   11.2s
[Parallel(n_jobs=128)]: Done 118 out of 248 | elapsed:    6.8s remaining:    7.5s
[Parallel(n_jobs=128)]: Done 143 out of 248 | elapsed:    6.9s remaining:    5.0s
[Parallel(n_jobs=128)]: Done 168 out of 248 | elapsed:    6.9s remaining:    3.3s
[Parallel(n_jobs=128)]: Done 193 out of 248 | elapsed:    7.0s remaining:    2.0s
[Parallel(n_jobs=128)]: Done 218 out of 248 | elapsed:    7.1s remaining:    1.0s
[Parallel(n_jobs=128)]: Done 243 out of 248 | elapsed:    7.2s remaining:    0.1s
[Parallel(n_jobs=128)]: Done 

Writing 10m_step1 data to Zarr store...


[Parallel(n_jobs=128)]: Using backend LokyBackend with 128 concurrent workers.
[Parallel(n_jobs=128)]: Batch computation too fast (0.03575730323791504s.) Setting batch_size=2.
[Parallel(n_jobs=128)]: Done  18 out of 248 | elapsed:    0.1s remaining:    1.2s
[Parallel(n_jobs=128)]: Done  43 out of 248 | elapsed:    0.2s remaining:    1.0s
[Parallel(n_jobs=128)]: Done  68 out of 248 | elapsed:    0.3s remaining:    0.8s
[Parallel(n_jobs=128)]: Done  93 out of 248 | elapsed:    0.4s remaining:    0.6s
[Parallel(n_jobs=128)]: Done 118 out of 248 | elapsed:    0.5s remaining:    0.5s
[Parallel(n_jobs=128)]: Done 143 out of 248 | elapsed:    0.6s remaining:    0.4s
[Parallel(n_jobs=128)]: Done 168 out of 248 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=128)]: Done 193 out of 248 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=128)]: Done 218 out of 248 | elapsed:    0.8s remaining:    0.1s
[Parallel(n_jobs=128)]: Done 243 out of 248 | elapsed:    0.9s remaining:    0.0s
[Par

Writing 10m_step2 data to Zarr store...


...

Writing remaining_height_step1 data to Zarr store...


[Parallel(n_jobs=128)]: Using backend LokyBackend with 128 concurrent workers.
[Parallel(n_jobs=128)]: Batch computation too fast (0.034578800201416016s.) Setting batch_size=2.
[Parallel(n_jobs=128)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=128)]: Done  32 tasks      | elapsed:    0.1s
[Parallel(n_jobs=128)]: Done  57 tasks      | elapsed:    0.3s
[Parallel(n_jobs=128)]: Done  82 tasks      | elapsed:    0.3s
[Parallel(n_jobs=128)]: Done 109 tasks      | elapsed:    0.5s
[Parallel(n_jobs=128)]: Done 136 tasks      | elapsed:    0.5s
[Parallel(n_jobs=128)]: Done 165 tasks      | elapsed:    0.7s
[Parallel(n_jobs=128)]: Done 194 tasks      | elapsed:    0.8s
[Parallel(n_jobs=128)]: Done 225 tasks      | elapsed:    0.9s
[Parallel(n_jobs=128)]: Done 256 tasks      | elapsed:    1.0s
[Parallel(n_jobs=128)]: Done 322 tasks      | elapsed:    1.2s
[Parallel(n_jobs=128)]: Done 388 tasks      | elapsed:    1.4s
[Parallel(n_jobs=128)]: Done 458 tasks      | elapsed:    1.6s
[Par

Writing remaining_height_step2 data to Zarr store...


[Parallel(n_jobs=128)]: Using backend LokyBackend with 128 concurrent workers.
[Parallel(n_jobs=128)]: Batch computation too fast (0.04097151756286621s.) Setting batch_size=2.
[Parallel(n_jobs=128)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=128)]: Done  32 tasks      | elapsed:    0.2s
[Parallel(n_jobs=128)]: Done  57 tasks      | elapsed:    0.3s
[Parallel(n_jobs=128)]: Done  82 tasks      | elapsed:    0.3s
[Parallel(n_jobs=128)]: Done 109 tasks      | elapsed:    0.5s
[Parallel(n_jobs=128)]: Done 136 tasks      | elapsed:    0.6s
[Parallel(n_jobs=128)]: Done 165 tasks      | elapsed:    0.7s
[Parallel(n_jobs=128)]: Done 194 tasks      | elapsed:    0.8s
[Parallel(n_jobs=128)]: Done 225 tasks      | elapsed:    0.9s
[Parallel(n_jobs=128)]: Done 256 tasks      | elapsed:    1.0s
[Parallel(n_jobs=128)]: Done 322 tasks      | elapsed:    1.2s
[Parallel(n_jobs=128)]: Done 388 tasks      | elapsed:    1.5s
[Parallel(n_jobs=128)]: Done 458 tasks      | elapsed:    1.7s
[Para

OSError: [Errno -101] NetCDF: HDF error: '/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500/2015/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_2015_12.nc'

-   For now, the wind speed is stored into zarr.
-   However, we are not sure whether all the chunks are correctly stored or any zero chunks exist.
-   To check that, better load either entire year and find the zero chunks.
    -   This takes 8.9 minutes to load one year and then fraction of seconds to find the zero chunks.
-   Or, create a function to load 128 chunks in parallel and find zero chunks. 

In [24]:
# Open the Zarr store
store = zarr.open(zarr_store, mode="r")

# Select dimensions
time_dim = store["time"]
height_dim = store["heightAboveGround"]

# Specify the height level to check
height_level = 200  # Example: height 200

# Initialize the list to store all-zero time indices
all_zero_times = []

# Get the index of the specified height level
if height_level in height_dim[:]:
    h_idx = int(np.where(height_dim[:] == height_level)[0][0])
else:
    raise ValueError(f"Height level {height_level} not found in dataset.")

# Function to process a chunk
def process_chunk(time_idx):
    chunk = store["wind_speed"][time_idx, h_idx, :, :]  # Load the chunk
    if np.all(chunk == 0):
        print(time_idx)
        return time_idx
    return None

# Create tasks and execute them in parallel
with Parallel(n_jobs=128, verbose=10) as parallel:
    results = parallel(delayed(process_chunk)(t_idx) for t_idx in range(len(time_dim)))

# Filter out None results
all_zero_times = [r for r in results if r is not None]

# Output the indices of all-zero time chunks
print("All-zero chunks found at times:", all_zero_times)

[Parallel(n_jobs=128)]: Using backend LokyBackend with 128 concurrent workers.
[Parallel(n_jobs=128)]: Batch computation too fast (0.036696672439575195s.) Setting batch_size=2.
[Parallel(n_jobs=128)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=128)]: Done  32 tasks      | elapsed:    0.3s
[Parallel(n_jobs=128)]: Done  57 tasks      | elapsed:    0.3s
[Parallel(n_jobs=128)]: Done  82 tasks      | elapsed:    0.4s
[Parallel(n_jobs=128)]: Done 109 tasks      | elapsed:    0.4s
[Parallel(n_jobs=128)]: Done 136 tasks      | elapsed:    0.5s
[Parallel(n_jobs=128)]: Done 165 tasks      | elapsed:    0.5s
[Parallel(n_jobs=128)]: Done 194 tasks      | elapsed:    0.6s
[Parallel(n_jobs=128)]: Done 225 tasks      | elapsed:    0.6s
[Parallel(n_jobs=128)]: Done 256 tasks      | elapsed:    0.7s
[Parallel(n_jobs=128)]: Done 322 tasks      | elapsed:    0.7s
[Parallel(n_jobs=128)]: Done 388 tasks      | elapsed:    0.8s
[Parallel(n_jobs=128)]: Done 458 tasks      | elapsed:    0.9s
[Par

All-zero chunks found at times: []


[Parallel(n_jobs=128)]: Done 87672 out of 87672 | elapsed:  1.0min finished
